# Search HERUS for observed lines in ALMA archival data
## - HERUS (Clements+2018) is a sample of 43 ULIRGs at z < 0.3
## Search the ALMA archive using the `ALMAxmatch` module. Specifically we need the `runQueryWithLines` method

In [1]:
from astropy.table import Table
# read in sample
HERUS_sample = Table.read('HERUS_sample.fits')

Tidy up trailing white space in the name column (helps with matching later)

In [2]:
names = [name.strip() for name in HERUS_sample['name']]
HERUS_sample['name'] = names

In [3]:
HERUS_sample

name,ra,dec,z,log(LIR/L),ObsID_C18,F250,F350,F500,F250_err,F350_err,F500_err,Temp(K),T+,T-,Beta,B+,B-,A,A+,A-,DustMass(logMsol),HERUS
str15,float64,float64,float32,float32,bytes11,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes6,float32,float32,float32,bool
IRAS 00397-1312,10.5647,-12.93412,0.262,12.97,1342234696,0.389,0.13,0.04,0.004,0.004,0.005,41.73,1.19,1.19,2.03,0.08,0.08,- 1.21,0.05,0.05,8.9,True
3C273,187.27792,2.05239,0.158,12.72,1342234882,0.437,0.633,0.994,0.004,0.004,0.005,nan,nan,nan,nan,nan,nan,,nan,nan,nan,True
IRAS 20100-4156,303.37308,-41.79303,0.13,12.63,1342230817,1.001,0.349,0.102,0.004,0.004,0.005,39.51,0.4,0.39,1.96,0.03,0.03,- 0.63,0.02,0.02,8.6,True
Mrk1014,29.95921,0.39462,0.163,12.61,1342237540,0.46,0.175,0.063,0.004,0.004,0.005,46.89,1.1,1.14,1.5,0.07,0.06,- 1.30,0.04,0.04,8.4,True
IRAS 03521+0028,58.6758,0.61761,0.152,12.56,1342239850,0.684,0.27,0.094,0.004,0.004,0.004,42.62,1.15,1.27,1.57,0.05,0.05,- 1.02,0.04,0.04,8.6,True
IRAS 03158+4227,49.80167,42.64111,0.134,12.55,1342226656,0.973,0.377,0.137,0.004,0.004,0.005,42.3,1.26,1.37,1.58,0.04,0.04,- 0.84,0.04,0.04,8.6,True
IRAS 16090-0139,242.91847,-1.78516,0.134,12.54,1342229565,1.067,0.404,0.116,0.004,0.004,0.005,37.5,0.43,0.44,1.84,0.03,0.03,- 0.60,0.02,0.02,8.7,True
Mrk231,194.05931,56.87368,0.042,12.49,1342201218a,5.618,2.011,0.615,0.019,0.008,0.008,36.87,0.41,0.43,1.89,0.02,0.02,0.26,0.02,0.02,8.3,True
IRAS 00188-0856,5.36051,-8.65722,0.128,12.47,1342234693,0.877,0.345,0.111,0.004,0.004,0.005,35.76,0.88,1.05,1.74,0.05,0.05,- 0.67,0.05,0.04,8.6,True


### Set path to custom `astroquery` module
This is a temporary hack to "use a version of ```astroquery``` that does not have a bug in the release date column. 
To get this set up run the following in terminal:

`git clone https://github.com/astropy/astroquery.git`

`cd astroquery`

`python setup.py build`

`python setup.py install`

replace `python` with `python3` if necessary

In [4]:
import sys
sys.path = ['/Users/thbrown/astroquery'] + sys.path

### Import the `ALMAxmatch` tool
requires adding to the python path for now

In [5]:
import sys
sys.path = ['/Users/thbrown/ALMA/think_tank_code/tools/archiveDev'] + sys.path
from ALMAxmatch import archiveSearch

## Run the archive search
1. Create list of tuples containing `SkyCoord` objects and search radii ie. (SkyCoord(ra,dec), searchRadius)*
2. Define the line rest frequencies (names are optional, output defaults to *line 0*, *line 1*, ...)
2. Pass tuple array along with lines of interest to `runQueries`, selecting only science data

### 1. `archiveSearch` takes either a tuple ('ra dec', 'radius') or lists of tuples.

In [6]:
from astropy import units as u
from astropy.coordinates import SkyCoord

# archive search impact parameter
searchradius = '20arcsec'

# SkyCoord object
herusCoords = [(SkyCoord(ra=ra*u.degree, dec=dec*u.degree), searchradius)
                   for ra, dec in zip(HERUS_sample['ra'], HERUS_sample['dec'])]

Generate target list in `herusLineQuery`

In [7]:
# search using SkyCoord array
herusLineQuery = archiveSearch(herusCoords)

display the target dictionary

In [8]:
display(herusLineQuery.targets)

{'coord=(10.5647 deg -12.93412 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (10.5647, -12.93412)>, '20arcsec'),
 'coord=(187.27792 deg 2.05239 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (187.27792, 2.05239)>, '20arcsec'),
 'coord=(303.37308 deg -41.79303 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (303.37308, -41.79303)>, '20arcsec'),
 'coord=(29.95921 deg 0.39462 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (29.95921, 0.39462)>, '20arcsec'),
 'coord=(58.6758 deg 0.61761 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (58.6758, 0.61761)>, '20arcsec'),
 'coord=(49.80167 deg 42.64111 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (49.80167, 42.64111)>, '20arcsec'),
 'coord=(242.91847 deg -1.78516 deg) radius=20arcsec': (<SkyCoord (ICRS): (ra, dec) in deg
      (242.91847, -1.78516)>, '20arcsec'),
 'coord=(194.05931 deg 56.87368 deg) radius=20arcsec': (<SkyCoord (ICRS): (r

### 2. Define lines of interest

Search for the following lines:

1. $^{12}$CO ($J=1-0$) @ 115.27120180 GHz rest frequency
2. $^{13}$CO ($J=1-0$) @ 109.78217340 GHz rest frequency
3. C$^{18}$O ($J=1-0$) @ 110.20135430 GHz rest frequency

In [9]:
# rest frame frequencies
rf_12CO_10 = 115.27120180 # 12CO J=1-0
rf_13CO_10 = 110.20135430 # 13CO J=1-0
rf_C18O_10 = 109.78217340 # C18O J=1-0

rest_frequencies = [rf_12CO_10, rf_13CO_10, rf_C18O_10]
line_names = ['12CO J=1-0', '13CO J=1-0', 'C18O J=1-0'] # column names for observed boolean flags 

Run the query, pulling only science data where lines are observed

### 3. Search using `runQueriesWithLines`

In [10]:
herusLineQuery.runQueriesWithLines(restFreqs=rest_frequencies
                                    , line_names=line_names
                                    , science=True)

KeyError: 'NED Redshift'

**Display the results**

In [ ]:
n=0 #counter
for targetname in herusLineQuery.targets.keys():
    if len(herusLineQuery.queryResults[targetname])>=1:
        n = n+1
        display(herusLineQuery.queryResults[targetname])

In [ ]:
print("galaxies with 12CO, 13CO, OR C18O:", n)

In [ ]:
k=0 #counter
for targetname in herusLineQuery.targets.keys():
    if len(herusLineQuery.queryResults[targetname])>=1:
        # select only galaxies with all three lines
        if ((True in herusLineQuery.queryResults[targetname]['12CO J=1-0']) & 
            (True in herusLineQuery.queryResults[targetname]['13CO J=1-0']) &
            (True in herusLineQuery.queryResults[targetname]['C18O J=1-0'])):
            k = k+1
            display(herusLineQuery.queryResults[targetname])


In [ ]:
print("galaxies with 12CO, 13CO, AND C18O:", k)